# <center>Commencer à utiliser CLTK</center>

<center>Avril 2022</center>

## Abordé dans ce chapitre

Dans ce chapitre, vous allez apprendre à : 
- récupérer un texte,
- comprendre les notions de *pipeline* de traitement, de document CLTK ou `Doc`, 
- utiliser un *pipeline* de traitement sur un texte,
- manipuler un texte au niveau du mot et aussi au niveau de la phrase.

## Introduction

## Récupérer un texte

Bien que CLTK permette de récupérer des corpus de textes pour chaque langue, nous allons commencer par traiter des données stockées en local. Nous pouvons retrouver ce qu'on va faire dans <a href="https://github.com/cltk/cltk/blob/master/notebooks/CLTK%20Demonstration.ipynb">le *notebook* sur le dépôt de CLTK</a>. Dans le dépôt actuel, nous mettons à disposition une collection de textes.
Regardons un extrait de Tite-Live.

Tout d'abord, nous avons besoin de récupérer le texte en le stockant dans une variable.

In [ ]:
with open("texts/lat-livy.txt") as f:
    livy_full = f.read()

Excellent ! Maintenant, regardons ce que nous avons en main.

In [ ]:
print("Extrait du texte :", livy_full[:200])
print("Nombre de caractères :", len(livy_full))
print("Nombre approximatif de token :", len(livy_full.split()))

Nous utilisons l'expression "nombre approximatif de tokens" parce que les tokens sont considérés comme quelque chose qui a une fonction syntaxique dans le texte. Cela signifie qu'un token n'est pas seulement un mot, mais aussi un signe de ponctuation par exemple. Nous utilisons le terme "approximatif" parce que la fonction `split()` transforme une chaîne de caractères en liste en considérant par défaut le caractère espace. En d'autres termes, le nombre exact de tokens est plus élevé puisque les signes de ponctuation peuvent être collés aux mots.


## Utiliser le *pipeline* de traitement de CLTK

CLTK est spécialement conçu pour le traitement de langues naturelles appliqué aux langues antiques et médiévales. Pour utiliser au mieux cette bibliothèque, nous avons d'abord besoin d'importer le *pipeline* mentionné.

In [ ]:
from cltk import NLP

Si le code ci-dessus a fonctionné sans générer d'erreur, alors cela signifie que nous avons correctement importé la classe NLP depuis CLTK. Cela nous permet de créer un *pipeline* de traitement de CLTK. Pour ce faire, nous avons cependant besoin de connaître la langue dans laquelle le texte a été écrit. Tite-Live était un auteur romain, la langue est donc le latin et son code est "lat".

In [ ]:
# Charger le pipline par défaut du latin
cltk_nlp = NLP(language="lat")

`NLP(language="lat")` est une variable qui contient le *pipeline* de traitement par défaut proposé par CLTK pour le latin. Il est possible de définir son propre *pipeline* de traitement. Il est courant d'utiliser une variable qui contient le nom `nlp` pour se rappeler facilement qu'on manipule un *pipeline* de traitement.

La sortie de cette cellule fournit des informations clés sur notre *pipeline*. Ça inclut les différents *pipelines* de traitement,
ou processus appliqués aux données d'entrée (le texte) :
- `LatinNormalizeProcess`
- `LatinStanzaProcess`
- `LatinEmbeddingsProcess`
- `StopsProcess`
- `LatinNERProcess`
- `LatinLexiconProcess`

Nous allons étudier chacun de processus ou *process* en profondeur dans ce *notebook*. Pour le moment, il suffit de comprendre qu'un *pipeline* de traitement est une suite ordonnée de processus. L'entrée du *pipeline* s'applique à du texte et la sortie est un document `Doc` propre à CLTK. Un processus se situant après un autre processus peut en dépendre, l'ordre est donc important.
Nous allons aborder l'ensemble des processus plus tard dans ce *notebook*. Pour l'instant, il faut comprendre que le texte qui est passé par une instance de la classe NLP passe à travers un certain nombre de processus dans le *pipeline*. L'ordre est important à respecter puisque des processus dépendent du résultat des processus précédents.

Si on veut enlever un processus du *pipeline* de traitements, on peut modifier directement l'attribut `cltk_nlp.pipeline.processes` en appliquant la méthode `pop()`. Si aucun argument n'est donné, alors le dernier élément est retiré. S'il y a un argument, alors c'est l'indice de l'élément dans la liste à retirer.


In [ ]:
cltk_nlp.pipeline.processes.pop()
print(cltk_nlp.pipeline.processes)

Une raison de retirer un processus peut être de vouloir éviter des traitements trop long. Le processus `LatinLexiconProcess` prend beaucoup de temps et ne correspond pas forcément à tous les besoins, comme le nôtre qui est de faire un *pipeline* de traitements pour trouver les entités nommées dans des textes.


## The CLTK Doc Object

Maintenant que nous avons mis en place notre *pipeline*, analysons un texte. Pour ce faire, nous allons créer un objet CLTK Doc. Si vous êtes déjà familié avec spaCy ou d'autres bibliothèques de TAL, ça doit vous dire quelque chose. L'objet Doc contient les données du texte. Avant que nous examinions l'objet `Doc`, instancions en donc un. Tout d'abord, raccourcissons le texte de Tite-Live.


In [ ]:
livy = livy_full[:len(livy_full) // 12]
print("Nombre approximatif de tokens :", len(livy.split()))

Une fois le texte raccourci, créons un objet `Doc` en appelant la méthode `analyze()` de `cltk_doc` avec comme premier argument le texte à analyser. Si c'est la première fois que vous exécutez ce code, le code va vous demander d'entrer "Y" pour télécharger les modèles.

In [ ]:
cltk_doc = cltk_nlp.analyze(text=livy)

Une fois les modèles téléchargés, notre *pipeline* de traitements exécute les processus à partir du texte. Commençons par examiner l'objet `Doc` généré.

In [ ]:
print(type(cltk_doc))

C'est une classe définie par le code de CLTK. Ça ne fait pas partie du code de base de Python.

## Les accesseurs de l'objet Doc

L'objet `Doc` a des variables que l'on appelle attributs ou propriétés, selon qu'elles sont directement enregistrées comme telles (attributs) ou calculéss à partir d'attributs (propriétés). On regroupe les attributs et les propriétés sous le terme d'accesseurs (ou *accessors* en anglais). Dans notre cas, ces accesseurs, seront ou biende type de base de Python ou bien seront composés d'objets propres à CLTK comme `Word` par exemple. Ces accesseurs nous aideront à parser de différentes manières, à notre guise, les variables de type `Doc`.

Regardons d'abord les accesseurs directement disponibles à partir d'une variable `Doc`.

In [ ]:
accessors = ([x for x in dir(cltk_doc) if not x.startswith("__")])
for a in accessors:
    print (a)

Examinons plusieurs de ces accesseurs en détail. Chaque accesseur a un titre de telle manière qu'il vous est possible de naviguer facilement entre les paragraphes.

### Raw

L'attribut `raw` (cru, brut en anglais) est le texte brut que l'on a donné en entré du *pipeline*.

In [ ]:
print (cltk_doc.raw[:20])

### Tokens

L'attribut *tokens* est une liste ordonnée qui contient les tokens du texte.

In [ ]:
print(cltk_doc.tokens[:20])

Vous pouvez voir que les tokens sont isolés les uns par rapport aux autres dans une liste. Les mots et la ponctuation ont chacun leur place. Une étude du texte au niveau des mots est dès lors possible.

### Lemmata

Comme la propriété `tokens`, la propriété `lemmata` est aussi une liste dérivée du texte brut à la différence près que les éléments sont les lemmes associés aux tokens. Un lemme est la forme du dictionnaire associé à un token. Ici, on a le token "capta" qui est associé au lemme "capio" parce qu'en latin, la forme du dictionnaire d'un participe passé est la première personne du singulier du présent de l'indicatif du verbe en question.

In [ ]:
print(cltk_doc.lemmata[:20])

### POS

La propriété `pos` fonctionne de la même manière, mais contient la nature du mot en lieu et place du token dans `tokens`. `pos` est l'acronyme de *part-of-speech* ou *pars oratori* en latin qui correspond au français "nature du mot" ou "nature grammaticale". Cet attribut est très courant dans les bibliothèques de TAL.

In [ ]:
print(cltk_doc.pos[:20])

### Words

La propriété `words` est aussi une liste comme `tokens` ou `lemmata` mais elle ne contient pas de *strings* mais contient des objets de type `Word`. Regardons le septième élements de `words`.

In [ ]:
print (cltk_doc.words[6])

C'est un objet qui a lui-même plusieurs accesseurs ! Nous pouvons voir toutes les données pertinentes à l'échelle d'un mot. On peut voir quelle est la nature du mot sélectionné.

In [ ]:
print (cltk_doc.words[6].pos)

Maintenant que nous savons que c'est un verbe, on peut vouloir connaître sa voix (voix active ou voix passive). On regarde pour ça l'attribut `features` qui est un dictionnaire.

In [ ]:
print (cltk_doc.words[6].features)

Le dictionnaire a une clef "Voice" qui est associée à la valeur "passive". Ce verbe est à la voix passive !

In [ ]:
print (cltk_doc.words[6].features["Voice"])

On peut voir les autres élements facilement.

In [ ]:
print("Number:", cltk_doc.words[6].features["Number"])
print("Tense:", cltk_doc.words[6].features["Tense"])
print("VerbForm:", cltk_doc.words[6].features["VerbForm"]) 
print("Voice:", cltk_doc.words[6].features["Voice"])

Je vous encourage à prendre le temps de découvrir les possibilités offertes par les classes `Doc` et `Word` et leurs accesseurs. Grâce à elles, vous pouvez aisément profiter de toute la puissance de CLTK. En prime, il est possible d'ajouter autant d'accesseurs que nécessaires.

### Sentence Tokens

Contrairement aux attributs précédents, `sentence_tokens` nous permet d'analyser le texte au niveau de la phrase. Ainsi, le parsage phrase par phrase devient possible. L'approche la plus simple pour séparer les phrases est d'utiliser `split(".")`. Cependant, cette méthode est très imprécise parce que les points peuvent noter des abbréviations en plein milieu d'une phrase par exemple. Grâce aux fonctions *ad hoc* écrites par CLTK, on peut séparer aisément les phrases d'un texte en latin.

## Conclusion

Ce chapitre a présenté les principales caractéristiques de la class NLP et nous avons vu comment construire le *pipeline* de traitement et comment y passer un texte à travers lui. Dans le prochain chapitre, nous allons examiner plus particulièrement la reconnaissance d'entités nommées.